In [ ]:
from glob import glob
import json
import pandas as pd
from itertools import product
import numpy as np
from scipy.optimize import least_squares
from functools import partial

In [ ]:
from utils import make_countmatrix, sort_schools, sigmoid, logit, compute_p, compute_mu, to_np, cost

In [ ]:
import matplotlib.pyplot as plt
import json

data_dir = '../data/processed/'
with open(data_dir + 'stimulus_wr.json') as fh:
    stimulus_wr = json.load(fh)

for vp in [0, 1, 2, 4]:
    for g in ['low', 'medium', 'high']:
        name = data_dir + f'exp_processed_vp-{vp}_g-{g}.pkl'
        data = pd.read_pickle(name)

        p = dict((data.win1 / (data.win1 + data.win2)))
        p = dict(compute_p(data))
        p = dict(filter(lambda x: ~np.isnan(x[1]), p.items()))

        school2idx = {y: x for x, y in enumerate(set(y for x in data.index for y in x))}
        A = make_countmatrix(data, school2idx)
        schools_sorted = sort_schools(A, school2idx)
        # rev_schools_sorted = list(reversed(schools_sorted))

        # optimized_mu = least_squares(cost, to_np(mu_1)).x
        N = len(schools_sorted)
        f = partial(cost, schools_sorted, p)
        optimized_mu = least_squares(f, np.zeros(N)).x
        score = {city: optimized_mu[i] for i, city in enumerate(schools_sorted[:-1])}
        score[schools_sorted[-1]] = 0

        # plt.plot([stimulus_wr[key] for key in score], score.values(), 'x')
        plt.title(name)
        for key, value in score.items():
            plt.plot(stimulus_wr[key], value, 'x', label=key)
        plt.legend()
        plt.show()